## Preprocessing

In [35]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [36]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
## Drop other columns that won't be used to create factors
## special considerations removed as part of optimization process to try get more accurary
unwanted_columns = ["EIN", "NAME", "USE_CASE", "STATUS", "ASK_AMT", "SPECIAL_CONSIDERATIONS"]

application_df = application_df.drop(unwanted_columns, axis=1)

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,Association,0,1
1,T3,Independent,C2000,Co-operative,1-9999,1
2,T5,CompanySponsored,C3000,Association,0,0
3,T3,CompanySponsored,C2000,Trust,10000-24999,1
4,T3,Independent,C1000,Trust,100000-499999,1


In [37]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE    17
AFFILIATION          6
CLASSIFICATION      71
ORGANIZATION         4
INCOME_AMT           9
IS_SUCCESSFUL        2
dtype: int64

In [38]:
# Choose a cutoff value and create a list of application types to be replaced
for app in application_df["APPLICATION_TYPE"].value_counts().index[10:]:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Choose a cutoff value and create a list of affiliation to be replaced
for app in application_df["AFFILIATION"].value_counts().index[2:]:
    application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(app,"Other")

# Choose a cutoff value and create a list of classification to be replaced
for app in application_df["CLASSIFICATION"].value_counts().index[8:]:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(app,"Other")

# Choose a cutoff value and create a list of organization to be replaced
for app in application_df["ORGANIZATION"].value_counts().index[2:]:
    application_df['ORGANIZATION'] = application_df['ORGANIZATION'].replace(app,"Other")

# Choose a cutoff value and create a list of income_amt to be replaced
for app in application_df["INCOME_AMT"].value_counts().index[4:]:
    application_df['INCOME_AMT'] = application_df['INCOME_AMT'].replace(app,"Other")

## removed as factor to try increase accurary of model

# Choose a cutoff value and create a list of special_considerations to be replaced
#for app in application_df["SPECIAL_CONSIDERATIONS"].value_counts().index[2:]:
    #application_df['SPECIAL_CONSIDERATIONS'] = application_df['SPECIAL_CONSIDERATIONS'].replace(app,"Other")


In [39]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

application_df.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,CLASSIFICATION_C7000,CLASSIFICATION_Other,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_Other
0,1,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0


In [40]:
# Split our preprocessed data into our features and target arrays
target = application_df["IS_SUCCESSFUL"]
features = application_df.drop("IS_SUCCESSFUL", axis= 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state= 1)

In [41]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [58]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
feature_num = len(features.columns)

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=feature_num))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu", input_dim=feature_num))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="tanh", input_dim=feature_num))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units=20, activation="relu", input_dim=feature_num))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 80)                2560      
                                                                 
 dense_25 (Dense)            (None, 30)                2430      
                                                                 
 dense_26 (Dense)            (None, 30)                930       
                                                                 
 dense_27 (Dense)            (None, 20)                620       
                                                                 
 dense_28 (Dense)            (None, 1)                 21        
                                                                 
Total params: 6561 (25.63 KB)
Trainable params: 6561 (25.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [59]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [60]:
# Create checkpoints
from keras.src.saving.saving_lib import save_weights_only
from keras.src.saving.saving_api import save_weights
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs('OppCheckpoints/',exist_ok = True)
file_path = "OppCheckpoints/weights.{epoch:02d}.hdf5"

callback = ModelCheckpoint(filepath = file_path, save_weights_only = True, period = 5)

In [61]:
# Train the model
model_fit = nn.fit(X_train_scaled, y_train, epochs=100, verbose=1, callbacks=[callback])

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5691 - accuracy: 0.7171
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5577 - accuracy: 0.7255
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5551 - accuracy: 0.7268
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5543 - accuracy: 0.7262
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5539 - accuracy: 0.7280
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5529 - accuracy: 0.7267
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7287
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7274
Epoch 9/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5518 - accuracy: 0.7289
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accura

In [57]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5600 - accuracy: 0.7289 - 512ms/epoch - 2ms/step
Loss: 0.5600180625915527, Accuracy: 0.728863000869751


In [62]:
# Export our model to HDF5 file
nn.save("Data/alphebetsoupcharity_Optimization.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [63]:
# Download h3 file
from google.colab import files

files.download("Data/alphebetsoupcharity_Optimization.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>